In [1]:
import cv2 as cv
import numpy as np
import pandas as pd

import os
import sys
import csv

import nibabel as nib 
import imageio 
from PIL import Image
import csv

## For train data

In [ ]:
#find the mask of the RV into a binary mask and write it to Mask_RV
dirname = "./Train_img"

for root,dirs,files in os.walk(dirname):
    for file in files:
        s = root[-3:]
        if s == 'seg':
          img_path = os.path.join(root,file)
          print(img_path)
          img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
          for i in range(img.shape[0]):
            for j in range(img.shape[1]):
              if img[i][j] == 255 : #255 corresponding to 01, RV
                img[i][j] = 255
              else:
                img[i][j] = 0
          result_path = os.path.join('./Mask_RV',root[-10:])
          if not os.path.exists(result_path):
              os.mkdir(result_path)
          imageio.imwrite(os.path.join(result_path,file), img) #write to mask_rv/00x_Ex_seg folder, range:0-255
          mask = Image.open(os.path.join(result_path,file))
          mask.save(os.path.join(result_path,file))
        else:
          continue


In [ ]:
#find the mask of the myocardium into a binary mask and write it to Mask_Myocardium
dirname = "./Train_img"

for root,dirs,files in os.walk(dirname):
    for file in files:
        s = root[-3:]
        if s == 'seg': 
          img_path = os.path.join(root,file)
          print(img_path)
          img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
          for i in range(img.shape[0]):
            for j in range(img.shape[1]):
              if img[i][j] == 254 : #254 corresponding to 02 myocardium
                img[i][j] = 255
              else:
                img[i][j] = 0
          result_path = os.path.join('./Mask_Myocardium',root[-10:])
          if not os.path.exists(result_path):
              os.mkdir(result_path)
          imageio.imwrite(os.path.join(result_path,file), img) #write to mask_Myocardium/00x_Ex_seg folder, range:0-255
          mask.save(os.path.join(result_path,file))
        else:
          continue

In [ ]:
#calculate LV volume in ED and ES periods, respectively
dirname = "./Mask_LV"

Volume_ED_LV = []
Volume_ES_LV = []
for root in os.listdir(dirname):
        print(root)
        s = root[-6:-4]
        suf_ed = 0
        suf_es = 0
        print(s)
        root = os.path.join(dirname,root)
        if s == 'ED':
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_ed += np.sum(img)
            Volume_ED_LV.append(suf_ed)
        else: #s == 'ES'
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_es += np.sum(img)
            Volume_ES_LV.append(suf_es)






Volume_ED_LV = np.array(Volume_ED_LV)
Volume_ES_LV = np.array(Volume_ES_LV)
Volume_ES_LV.shape


In [ ]:
#calculate RV volume in ED and ES periods, respectively
dirname = "./Mask_RV"

Volume_ED_RV = []
Volume_ES_RV = []
for root in os.listdir(dirname):
        print(root)
        s = root[-6:-4]
        suf_ed = 0
        suf_es = 0
        print(s)
        root = os.path.join(dirname,root)
        if s == 'ED':
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_ed += np.sum(img)
            Volume_ED_RV.append(suf_ed)
        else: #s == 'ES'
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_es += np.sum(img)
            Volume_ES_RV.append(suf_es)






Volume_ED_RV = np.array(Volume_ED_RV)
Volume_ES_RV = np.array(Volume_ES_RV)
Volume_ES_RV.shape

In [ ]:
#calculate Myocardiumvolume in ED and ES periods, respectively

dirname = "./Mask_Myocardium"

Volume_ED_Mc = []
Volume_ES_Mc = []
for root in os.listdir(dirname):
        print(root)
        s = root[-6:-4]
        suf_ed = 0
        suf_es = 0
        print(s)
        root = os.path.join(dirname,root)
        if s == 'ED':
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_ed += np.sum(img)
            Volume_ED_Mc.append(suf_ed)
        else: #s == 'ES'
            for img_num in os.listdir(root):
                img_path = os.path.join(root,img_num)
                img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
                img = img/255
                suf_es += np.sum(img)
            Volume_ES_Mc.append(suf_es)






Volume_ED_Mc = np.array(Volume_ED_Mc)
Volume_ES_Mc = np.array(Volume_ES_Mc)
Volume_ES_Mc.shape

In [7]:
# ratio between RV/LV at ED ES
Ratio_RV_LV_ED = Volume_ED_RV/Volume_ED_LV
Ratio_RV_LV_ES = Volume_ES_RV/Volume_ES_LV

# ratio between Mc/LV at ED ES
Ratio_Mc_LV_ED = Volume_ED_Mc/Volume_ED_LV
Ratio_Mc_LV_ES = Volume_ES_Mc/Volume_ES_LV

# the LV and RV ejection fraction
EF_LV = (Volume_ED_LV - Volume_ES_LV)/ Volume_ED_LV
EF_RV = (Volume_ED_RV - Volume_ES_RV)/ Volume_ED_RV

In [ ]:
#write the volumes and ratios to DataTrain.csv
import csv
feature = [Volume_ED_LV,Volume_ES_LV,Volume_ED_RV,Volume_ES_RV,Volume_ED_Mc,Volume_ES_Mc,
            EF_LV,EF_RV,Ratio_RV_LV_ED,Ratio_RV_LV_ES,Ratio_Mc_LV_ED,Ratio_RV_LV_ES]
name = ['Volume_ED_LV','Volume_ES_LV','Volume_ED_RV','Volume_ES_RV','Volume_ED_Mc','Volume_ES_Mc',
            'EF_LV','EF_RV','Ratio_RV_LV_ED','Ratio_RV_LV_ES','Ratio_Mc_LV_ED','Ratio_RV_LV_ES']

for i in range(len(feature)):
    print(i)
    new_col = [[x] for x in feature[i]]  

    with open('DataTrain.csv', 'r') as infile:
        reader = csv.reader(infile)
        rows = list(reader)

    for j in range(len(rows)-1):
        rows[j+1].insert(4+i, new_col[j][0])

    #column_name = name[i]
    rows[0].insert(4+i, name[i])


    with open('DataTrain.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)
